# Metody obliczeniowe w nauce i technice

## Laboratorium 4 - Singular Value Decomposition

### Sprawozdanie sporządził: Marcin Zielonka

### Wstęp

Do realizacji zadań skorzystam z gotowych funkcjonalności zawartych w bibliotekach:
* `numpy`
* `math`
* `nltk`
* `num2words`
* `re`
* `scipy`
* `functools`

In [1]:
import numpy as np
import re
import math
import scipy

from num2words import num2words

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

from functools import reduce

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Zadanie 1: Wyszukiwarka

1. Przygotuj duży (> 1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomocą *Web crawlera*, zbiór rozdziałów wyciętych z różnych książek)

Do realizacji zadań wykorzystam zbiór 1100 wiadomości tekstowych w języku angielskim (źródło: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). Każda wiadomość umieszczona jest w osobnej linii w pliku `data.txt`.

In [2]:
file = open('data.txt', 'r')

documents = np.array(list(file))

print(f'Number of documents: {len(documents)}')

Number of documents: 1100


2. Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów cech *bag-of-words* (indeksacja). Przykładowo zbiorem takim może być unia wszystkich słów występujących we wszystkich tekstach.

Aby wyszukiwarka działała wydajnie, każdy z tekstów został wstępnie odpowiednio przetworzony:

* zamieniono wszystkie litery na małe
* usunięto znaki specjalne
* zamieniono liczby na ich odpowiedniki w jęz. angielskim
* usunięto powtarzające się spacje
* usunięto słowa pełniące jedynie rolę pomocniczą (np. aby zachować reguły gramatyki) - takie jak *a*, *the*, itd.

Do tego celu wykorzystano *regular expressions* oraz gotowe funkcjonalności zawarte w bibliotekach `nltk` oraz `num2words`.

In [3]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [4]:
def prepare_document(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub('_', '', text)
    text = re.sub(' +', ' ', text)
    
    words = []
    
    for word in text.split():
        word = lemmatizer.lemmatize(word, pos='v')
        
        try:
            word = num2words(int(word))
        except ValueError:
            pass
        
        if word in stop_words:
            continue
        
        words.append(word)
    
    return words

In [5]:
def make_dictionary(documents):
    dict = {}
    
    all_words = reduce(list.__add__, list(map(prepare_document, documents)))
    
    for word in all_words:
        if word not in dict.keys():
            dict[word] = 0
        dict[word] += 1
    
    all_words = list(dict.keys())
    all_words.sort()
    
    return dict, all_words

In [6]:
dictionary, all_words = make_dictionary(documents)

3. Dla każdego dokumentu $j$ wyznacz wektor cech *bag-of-words* $\mathbf{d}_j$ zawierający częstości występowania poszczególnych słów (termów) w tekście.

In [7]:
def bag_of_words(words, all_words):
    bag_of_words = [0] * len(all_words)
    words_amount = len(words)
    
    for word in words:
        idx = all_words.index(word)
        bag_of_words[idx] += (1 / words_amount)

    return bag_of_words

In [8]:
def bags_of_words(documents, all_words):
    return list(map(lambda words: bag_of_words(words, all_words), list(map(prepare_document, documents))))

In [9]:
bags_of_words = bags_of_words(documents, all_words)

4. Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory cech ułożone są kolumnowo $A_{m×n}=[\mathbf{d}_1|\mathbf{d}_2|...|\mathbf{d}_n]$ ($m$ jest liczbą termów w słowniku, a $n$ liczbą dokumentów)

In [10]:
def term_by_document_matrix(bags_of_words):
    matrix = []
    
    for bag in bags_of_words:
        matrix.append(bag)

    return matrix
        

In [11]:
term_by_document_matrix = term_by_document_matrix(bags_of_words)


5. Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy *bag-of-words* przez *inverse document frequency*. Operacja ta pozwoli na redukcję znaczenia często występujących słów.

$$IDF(w)=log\frac{N}{n_w}$$

   gdzie $n_w$ jest liczbą dokumentów, w których występuje słowo $w$, a $N$ jest całkowitą
liczbą dokumentów.

In [26]:
def multiply_by_idf(term_by_document_matrix):
    documents_size = len(term_by_document_matrix)
    words_frequency = []
    
    for idx in range(len(term_by_document_matrix[0])):
        occurences = 0
        
        for doc in term_by_document_matrix:
           if doc[idx] > 0:
            occurences += 1
        
        words_frequency.append(math.log(documents_size / occurences))
        
    IDF = []
    words_frequency = np.array(words_frequency)
    for doc in np.array(term_by_document_matrix):
        IDF.append(doc * words_frequency)
    
    return IDF

In [30]:
IDF = multiply_by_idf(term_by_document_matrix)

6. Napisz program pozwalający na wprowadzenie zapytania (w postaci sekwencji słów) przekształcanego następnie do reprezentacji wektorowej $\mathbf{q}$ (*bag-of-words*). Program ma zwrócić $k$ dokumentów najbardziej zbliżonych do podanego zapytania $\mathbf{q}$. Użyj korelacji między wektorami jako miary podobieństwa.

$$
cos\theta_j=
\frac{\mathbf{q}^T\mathbf{d}_j}{\lVert\mathbf{q}\rVert\lVert\mathbf{d}_j\rVert}=
\frac{\mathbf{q}^T\mathbf{A}\mathbf{e}_j}{\lVert\mathbf{q}\rVert\lVert\mathbf{A}\mathbf{e}_j\rVert}
$$

In [14]:
def query(search_text, k, IDF, all_words):
    number_of_words = len(all_words)
    
    search_words = prepare_document(search_text)
    
    q = np.array(bag_of_words(search_words, all_words))
    q_norm = np.linalg.norm(q)
    
    result = []
    
    for idx, doc in enumerate(np.array(IDF)):
        doc_val = (q.T @ doc) / (q_norm * np.linalg.norm(doc))
        result.append((idx, doc_val))
    
    result.sort(key=lambda doc_tuple: doc_tuple[1], reverse=True)
    
    for doc_idx in [doc[0] for doc in result[:k]]:
        print(f'Message {doc_idx}:')
        print(documents[doc_idx])
    

In [15]:
query("dinner tonight?", 5, IDF, all_words)

Message 73:
I'm really not up to it still tonight babe

Message 333:
Huh so late... Fr dinner?

Message 498:
Huh so early.. Then ü having dinner outside izzit?

Message 243:
K, I might come by tonight then if my class lets out early

Message 458:
Probably gonna be here for a while, see you later tonight &lt;)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


7. Zastosuj normalizację wektorów cech $\mathbf{d}_j$ i wektora $\mathbf{q}$, tak aby miały one długość 1. Użyj zmodyfikowanej miary podobieństwa otrzymując

$$|\mathbf{q}^T\mathbf{A}|=[|cos\theta_1|,|cos\theta_2|,...,|cos\theta_n|]$$

In [16]:
def query_norm(search_text, k, term_by_document_matrix, all_words):
    number_of_words = len(all_words)
    
    search_words = prepare_document(search_text)
    
    q = np.array(bag_of_words(search_words, all_words))
    q = np.array([q / np.linalg.norm(q)])
    
    A = np.array([e / np.linalg.norm(e) for e in term_by_document_matrix])
    
    M = A @ q.T
    result = []
    
    for idx, val in enumerate(M):
        result.append((idx, val))
    
    result.sort(key=lambda doc_tuple: doc_tuple[1], reverse=True)
    
    for doc_idx in [doc[0] for doc in result[:k]]:
        print(f'Message {doc_idx}:')
        print(documents[doc_idx])

In [31]:
query_norm("sorry battery dead", 5, term_by_document_matrix, all_words)

Message 198:
Sorry battery died, yeah I'm here

Message 761:
I wonder if your phone battery went dead ? I had to tell you, I love you babe

Message 768:
I am sorry it hurt you.

Message 381:
Sorry, my battery died, I can come by but I'm only getting a gram for now, where's your place?

Message 557:
I'm gonna say no. Sorry. I would but as normal am starting to panic about time. Sorry again! Are you seeing on Tuesday?



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


8. W celu usunięcia szumu z macierzy A zastosuj SVD i low rank approximation otrzymując

$$
\mathbf{A}\approx\mathbf{A}_k=
\mathbf{U}_k\mathbf{D}_k\mathbf{V}_k^T=
[\mathbf{u}_1|...|\mathbf{u}_k]
\begin{bmatrix}
\sigma_1 & & \\
& \ddots & \\
& & \sigma_k
\end{bmatrix}
\begin{bmatrix}
\mathbf{v}_1^T \\
\vdots \\
\mathbf{v}_k^T
\end{bmatrix}
=
\sum_{i=1}^{k}\sigma_i\mathbf{u}_i\mathbf{v}_i^T
$$

oraz nową miarę podobieństwa

$$
cos\Phi_j=
\frac{\mathbf{q}^T\mathbf{A}_k\mathbf{e}_j}{\lVert\mathbf{q}\rVert\lVert\mathbf{A}_k\mathbf{e}_j\rVert}
$$

In [18]:
def low_rank_approximation(IDF, k):
    U, S, V = scipy.sparse.linalg.svds(np.array(IDF), k=k)
    return U @ np.diag(S) @ V

In [19]:
lr_approximations = []
for k in range(5,101,5):
    lr_approximations.append(low_rank_approximation(IDF, k))

9. Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. Dla jakiej wartości $k$ wyniki wyszukiwania są najlepsze (subiektywnie). Zbadaj wpływ przekształcenia IDF na wyniki wyszukiwania.

In [35]:
print('Results without removing noise:\n')
query("sorry battery dead", 3, IDF, all_words)

Results without removing noise:

Message 768:
I am sorry it hurt you.

Message 289:
Sorry, I'll call later

Message 65:
Sorry, I'll call later



In [34]:
print('Results with removing noise:\n\n')

for idx, IDF in enumerate(lr_approximations):
    print(f'=============== k: {idx * 5 + 5} ===============')
    query("sorry battery dead", 3, IDF, all_words)

Results with removing noise:


=============== k: 5 ===============
Message 117:
Sir, Waiting for your mail.

Message 59:
U can call me now...

Message 385:
10 min later k...

=============== k: 10 ===============
Message 59:
U can call me now...

Message 66:
K. Did you call me just now ah?

Message 926:
Please da call me any mistake from my side sorry da. Pls da goto doctor.

=============== k: 15 ===============
Message 768:
I am sorry it hurt you.

Message 534:
sorry, no, have got few things to do. may be in pub later.

Message 904:
That's fine, I'll bitch at you about it later then

=============== k: 20 ===============
Message 768:
I am sorry it hurt you.

Message 534:
sorry, no, have got few things to do. may be in pub later.

Message 904:
That's fine, I'll bitch at you about it later then

=============== k: 25 ===============
Message 768:
I am sorry it hurt you.

Message 904:
That's fine, I'll bitch at you about it later then

Message 534:
sorry, no, have got few things to do. 

Wyszukiwanie z wstępnym usuwaniem szumu daje bardzo zbliżone rezulaty do wyszukiwania bez usuwania szumu dla wartości $k$ powyżej wartości 15/20. Poniżej tej wartości otrzymane wyniki zdają się być losowe i często nie zawierają żadnych z wprowadzonych słów kluczowych.

Dla wyższych wartości $k$ (rzędu 60-70) algorytm przeszukiwania daje najbardziej zbliżone do wyszukiwania rezultaty.

#### Wpływ przekształcenia IDF na wyniki wyszukiwania.

In [33]:
query("sorry battery dead", 5, term_by_document_matrix, all_words)

Message 198:
Sorry battery died, yeah I'm here

Message 761:
I wonder if your phone battery went dead ? I had to tell you, I love you babe

Message 768:
I am sorry it hurt you.

Message 381:
Sorry, my battery died, I can come by but I'm only getting a gram for now, where's your place?

Message 557:
I'm gonna say no. Sorry. I would but as normal am starting to panic about time. Sorry again! Are you seeing on Tuesday?



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


Przekształcenie IDF ma znaczący wpływ na wyniki wyszukiwania, ponieważ dzięki tej transformacji wyszukiwarka jest bardziej wyczulona na rzadziej występujące słowa, a mniej na te popularniejsze (w tym przykładzie na słowo *sorry*). Co za tym idzie jest większe prawdopodobieństwo, że otrzymamy bardziej interesujące nas wyniki